##### Copyright 2025 Google LLC.

In [11]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Gemini API: Getting started with information grounding for Gemini models

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Grounding.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

In this notebook you will learn how to use information grounding with [Gemini models](https://ai.google.dev/gemini-api/docs/models/).

Information grounding is the process of connecting these models to specific, verifiable information sources to enhance the accuracy, relevance, and factual correctness of their responses. While LLMs are trained on vast amounts of data, this knowledge can be general, outdated, or lack specific context for particular tasks or domains. Grounding helps to bridge this gap by providing the LLM with access to curated, up-to-date information.

Here you will experiment with:
- Grounding information using <a href="#search_grounding">Google Search grounding</a>
- Adding <a href="#yt_links">YouTube links</a> to gather context information to your prompt
- Using <a href="#url_context">URL context</a> to include website, pdf or image URLs as context to your prompt

## Set up the SDK and the client

### Install SDK

This guide uses the [`google-genai`](https://pypi.org/project/google-genai) Python SDK to connect to the Gemini models.

In [12]:
# Grounding with Google Maps was introduced in 1.43
%pip install -q -U "google-genai>=1.43.0"

### Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [13]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

### Select model and initialize SDK client

Select the model you want to use in this guide, either by selecting one in the list or writing it down. Keep in mind that some models, like the 2.5 ones are thinking models and thus take slightly more time to respond (cf. [thinking notebook](./Get_started_thinking.ipynb) for more details and in particular learn how to switch the thiking off).

In [14]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

MODEL_ID = "gemini-2.5-flash"  # @param ["gemini-2.5-flash-lite", "gemini-2.5-flash-lite-preview-09-2025", "gemini-2.5-flash", "gemini-2.5-flash-preview-09-2025", "gemini-2.5-pro"] {"allow-input":true, isTemplate: true}

## Use Google Search grounding

<a name="search_grounding"></a>

Google Search grounding is particularly useful for queries that require current information or external knowledge. Using Google Search, Gemini can access nearly real-time information and better responses.

To enable Google Search, simply add the `google_search` tool in the `generate_content`'s `config` that way:
```
    config={
      "tools": [
        {
          "google_search": {}
        }
      ]
    },
```

In [15]:
from IPython.display import HTML, Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents="What was the latest Indian Premier League match and who won?",
    config={"tools": [{"google_search": {}}]},
)

# print the response
display(Markdown(f"**Response**:\n {response.text}"))
# print the search details
print(f"Search Query: {response.candidates[0].grounding_metadata.web_search_queries}")
# urls used for grounding
print(f"Search Pages: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")

display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

**Response**:
 The latest fully concluded Indian Premier League (IPL) match was the final of the IPL 2024 season, which took place on May 26, 2024.

In this match, the Kolkata Knight Riders (KKR) faced Sunrisers Hyderabad (SRH) at the M. A. Chidambaram Stadium in Chennai. Sunrisers Hyderabad won the toss and chose to bat first, scoring 113 runs in 18.3 overs. The Kolkata Knight Riders then successfully chased this target in 10.3 overs, winning by eight wickets.

Therefore, the **Kolkata Knight Riders** won the latest Indian Premier League match. This victory secured their third IPL title. Mitchell Starc of KKR was awarded Player of the Match.

Search Query: ['latest Indian Premier League match and winner', 'IPL 2024 final match winner']
Search Pages: wikipedia.org, iplt20.com


You can see that running the same prompt without search grounding gives you outdated information:

In [16]:
from IPython.display import Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents="What was the latest Indian Premier League match and who won?",
)

# print the response
display(Markdown(response.text))

The latest Indian Premier League match was the **final of the IPL 2024 season**, which took place on **May 26, 2024**.

**Teams:** Kolkata Knight Riders (KKR) vs. Sunrisers Hyderabad (SRH)
**Winner:** **Kolkata Knight Riders (KKR)** won by 8 wickets.

For more examples, please refer to the [dedicated notebook](./Search_Grounding.ipynb).

## Use Google Maps grounding

Google Maps grounding allows you to easily incorporate location-aware functionality into your applications. When a prompt has context related to Maps data, the Gemini model uses Google Maps to provide factually accurate and fresh answers that are relevant to the specified location or general area.

To enable grounding with Google Maps, add the `google_maps` tool in the  `config` argument of `generate_content`, and optionally provide a structured location in the `tool_config`.

```python
client.models.generate_content(
    ...,
    config=types.GenerateContentConfig(
      # Enable the tool.
      tools=[types.Tool(google_maps=types.GoogleMaps())],
      # Provide structured location.
      tool_config=types.ToolConfig(retrieval_config=types.RetrievalConfig(
            lat_lng=types.LatLng(
                latitude=34.050481, longitude=-118.248526))),
    )
)
```

In [17]:
from IPython.display import Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents="Do any cafes around here do a good flat white? I will walk up to 20 minutes away",
    config=types.GenerateContentConfig(
        tools=[types.Tool(google_maps=types.GoogleMaps())],
        tool_config=types.ToolConfig(
            retrieval_config=types.RetrievalConfig(
                lat_lng=types.LatLng(latitude=40.7680797, longitude=-73.9818957)
            )
        ),
    ),
)

Markdown(f"### Response\n {response.text}")

### Response
 Yes, there are several highly-rated cafes within a 20-minute walk that serve coffee, and some explicitly mention flat whites.

Here are a few options:
*   **Tiny Dancer Coffee** specifically mentions serving "espressos and flat whites" and has a rating of 4.8 stars. It's approximately a 6.8-minute walk from your current location (1.3 kilometers).
*   **787 coffee** specializes in espresso drinks and lattes and boasts a 4.9-star rating. It's about a 7.6-minute walk away (1.7 kilometers).
*   **Sote Coffee Roasters** has a 4.9-star rating and serves "freshly roasted brews," suggesting high-quality coffee. This is about a 5.9-minute walk (1.5 kilometers).
*   **White Noise Coffee - Coffee Shop & Roastery** has a 4.7-star rating and features "globally sourced beans, roasted in-house," which is often a sign of good coffee. It's approximately a 5.0-minute walk (1.2 kilometers).
*   **Cafe aroma** is very close, just a 1.6-minute walk (279 meters), and has a 4.7-star rating.

All grounded outputs require sources to be displayed after the response text. This code snippet will display the sources.

In [18]:
def generate_sources(response: types.GenerateContentResponse):
  grounding = response.candidates[0].grounding_metadata
  # You only need to display sources that were part of the grounded response.
  supported_chunk_indices = {i for support in grounding.grounding_supports for i in support.grounding_chunk_indices}

  sources = []
  if supported_chunk_indices:
    sources.append("### Sources from Google Maps")
  for i in supported_chunk_indices:
    ref = grounding.grounding_chunks[i].maps
    sources.append(f"- [{ref.title}]({ref.uri})")

  return "\n".join(sources)


Markdown(generate_sources(response))

### Sources from Google Maps
- [Le Cafe Coffee](https://maps.google.com/?cid=3766819750231249954)
- [White Noise Coffee - Coffee Shop & Roastery](https://maps.google.com/?cid=9563404650783060353)
- [Tiny Dancer Coffee](https://maps.google.com/?cid=14421445427760414557)
- [Heaven on 7th Marketplace](https://maps.google.com/?cid=13100894621228039586)

The response also includes data you can use to assemble in-line links. See the [Grounding with Google Search docs](https://ai.google.dev/gemini-api/docs/google-search#attributing_sources_with_inline_citations) for an example of this.

### Render the contextual Google Maps widget

If you are building a web-based application, you can add an interactive widget that includes a map view, the contextual location, the places Gemini considered in the query, and review snippets.

To load the widget, perform all of the following steps.
1. [Acquire a Google Maps API key](https://developers.google.com/maps/documentation/javascript/get-api-key), enabled for the Places API and the Maps JavaScript API,
1. Request the widget token in your request (with `GoogleMaps(enable_widget=True)`),
1. [Load the Maps JavaScript API](https://developers.google.com/maps/documentation/javascript/load-maps-js-api) and enable the Places library,
1. Render the [`<gmp-place-contextual/>`](https://developers.google.com/maps/documentation/javascript/reference/places-widget#PlaceContextualElement) element, setting `context-token` to the value of the `google_maps_widget_context_token` returned in the Gemini API response.

Note that generating a widget can add additional latency to the response, so it is recommended that you do not enable the widget if you are not displaying it.

Assuming you have a Google Maps API key with both APIs enabled, the following code shows one way to render the widget.

In [19]:

)

widget_token = response.candidates[0].grounding_metadata.google_maps_widget_context_token

display(Markdown(f"### Response\n {response.text}"))
display(Markdown(generate_sources(response)))
display(HTML(f"""
<!DOCTYPE html>
<html>
  <body>
    <div style="max-width: 500px; margin: 0 auto">
      <script src="https://maps.googleapis.com/maps/api/js?key={MAPS_API_KEY}&loading=async&v=alpha&libraries=places" async></script>
      <gmp-place-contextual context-token="{widget_token}"></gmp-place-contextual>
    </div>
  </body>
</html>
"""))

SecretNotFoundError: Secret MAPS_API_KEY does not exist.

Running and rendering the above code will require a Maps API key. Once you have it working, the widget will look like this.

![Rendered contextual Places widget](https://storage.googleapis.com/generativeai-downloads/images/maps-widget.png)

## Grounding with YouTube links

<a name="yt_links"></a>

You can directly include a public YouTube URL in your prompt. The Gemini models will then process the video content to perform tasks like summarization and answering questions about the content.

This capability leverages Gemini's multimodal understanding, allowing it to analyze and interpret video data alongside any text prompts provided.

You do need to explicitly declare the video URL you want the model to process as part of the contents of the request using a `FileData` part. Here a simple interaction where you ask the model to summarize a YouTube video:

In [20]:
yt_link = "https://www.youtube.com/watch?v=XV1kOFo1C8M"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=types.Content(
        parts=[
            types.Part(text="Summarize this video."),
            types.Part(file_data=types.FileData(file_uri=yt_link)),
        ]
    ),
)

Markdown(response.text)

The video introduces "Gemma Chess," a new application of Google's Gemma 3 language model designed to enhance the chess experience. Ju-yeong Ji from Google DeepMind explains that the goal isn't to replace powerful chess engines like AlphaZero or Stockfish, which excel at calculating optimal moves. Instead, Gemma aims to bring a "new dimension" to chess by leveraging its ability to understand and generate human-like text.

Key applications of Gemma in chess include:
1.  **Chess Explainer/Analysis:** Gemma can analyze chess games and provide understandable explanations for moves, strategies, and tactical ideas, rather than just raw engine evaluations. It can detail why a specific move is good, what the underlying strategic concepts are, and potential dangers, as demonstrated with a Kasparov vs. Deep Blue game.
2.  **Storytelling:** The model can turn chess game data (moves, player information, tournament details) into engaging narratives, bringing historical or personal matches to life and making them more interesting to review.
3.  **Personalized Learning:** Gemma acts as a "super helpful study buddy" or personal coach, explaining complex chess concepts (like the Sicilian Defense or passed pawns) in natural language, tailored to the user's skill level, and even in different languages (e.g., Korean). It can also provide feedback and suggest areas for improvement.

By combining the computational strength of traditional chess AI with Gemma's advanced linguistic abilities, this approach offers a more intuitive and human-centric way to learn, analyze, and appreciate the game of chess.

But you can also use the link as the source of truth for your request. In this example, you will first ask how Gemma models can help on chess games:

In [ ]:
yt_link = "https://www.youtube.com/watch?v=XV1kOFo1C8M"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=types.Content(
        parts=[
            types.Part(
                text="In 2 paragraph, how Gemma models can help on chess games?"
            ),
            types.Part(file_data=types.FileData(file_uri=yt_link)),
        ]
    ),
)

Markdown(response.text)

Now your answer is more insightful for the topic you want, using the knowledge shared on the video and not necessarily available on the model knowledge.

## Grounding information using URL context

<a name="url_context"></a>

The URL Context tool empowers Gemini models to directly access and process content from specific web page URLs you provide within your API requests. This is incredibly interesting because it allows your applications to dynamically interact with live web information without needing you to manually pre-process and feed that content to the model.

URL Context is effective because it allows the models to base its responses and analysis directly on the content of the designated web pages. Instead of relying solely on its general training data or broad web searches (which are also valuable grounding tools), URL Context anchors the model's understanding to the specific information present at those URLs.

### Process website URLs

If you want Gemini to specifically ground its answers thanks to the content of a specific website, just add the urls in your prompt and enable the tool by adding it to your config:
```
config = {
  "tools": [
    {
      "url_context": {}
    }
  ],
}
```

You can add up to 20 links in your prompt.

In [ ]:
prompt = """
  Based on https://ai.google.dev/gemini-api/docs/models, what are the key
  differences between Gemini 1.5, Gemini 2.0 and Gemini 2.5 models?
  Create a markdown table comparing the differences.
"""

config = {
    "tools": [{"url_context": {}}],
}

response = client.models.generate_content(
    contents=[prompt], model=MODEL_ID, config=config
)

display(Markdown(response.text))

You can see the status of the retrival using `url_context_metadata`:

In [ ]:
# get URLs retrieved for context
print(response.candidates[0].url_context_metadata)

### Add PDFs by URL

Gemini can also process PDFs from an URL. Here's an example:

In [ ]:
prompt = """
  Can you give me an overview of the content of this pdf?
  https://abc.xyz/assets/cc/27/3ada14014efbadd7a58472f1f3f4/2025q2-alphabet-earnings-release.pdf

"""

config = {
    "tools": [{"url_context": {}}],
}

response = client.models.generate_content(
    contents=[prompt], model=MODEL_ID, config=config
)

display(Markdown(response.text.replace("$", "\$")))

### Add images by URL

Gemini can also process images from an URL. Here's an example:

In [ ]:
prompt = """
  Can you help me name of the numbered parts of that instrument, in French?
  https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Trombone.svg/960px-Trombone.svg.png

"""

config = {
    "tools": [{"url_context": {}}],
}

response = client.models.generate_content(
    contents=[prompt], model=MODEL_ID, config=config
)

display(Markdown(response.text))

## Mix Search grounding and URL context

The different tools can also be use in conjunction by adding them both to the config. It's a good way to steer Gemini in the right direction and then let it do its magic using search grounding.

In [ ]:
prompt = """
  Can you give me an overview of the content of this pdf?
  https://abc.xyz/assets/cc/27/3ada14014efbadd7a58472f1f3f4/2025q2-alphabet-earnings-release.pdf
  Search on the web for the reaction of the main financial analysts, what's the trend?
"""

config = {
  "tools": [
      {"url_context": {}},
      {"google_search": {}}
  ],
}

response = client.models.generate_content(
  contents=[prompt],
  model=MODEL_ID,
  config=config
)

display(Markdown(response.text.replace('$','\$')))
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

## Next steps

<a name="next_steps"></a>

* For more details about using Google Search grounding, check out the [Search Grounding cookbook](./Search_Grounding.ipynb).
* If you are looking for another scenarios using videos, take a look at the [Video understanding cookbook](./Video_understanding.ipynb).

Also check the other Gemini capabilities that you can find in the [Gemini quickstarts](https://github.com/google-gemini/cookbook/tree/main/quickstarts/).